In [1]:
import zipfile
import os
import math
import pandas as pd
import cx_Oracle

## 初始时间设定

In [2]:
# 数据库信息
ana_db_user='NTTASK'
ana_db_pass='NTTASK'
ana_db='ESXVM0504'

# 分析日期
ana_date = 20200831
ana_mon = str(ana_date)[0:6]

In [3]:
def unzip_file(ana_date):
    # 解压缩文件
    province_file = '全省业务发展日报({}).zip'.format(ana_date)
    f_province_zip = zipfile.ZipFile(province_file)
    print('Extracting……'.format(f_province_zip.extract(member=f_province_zip.namelist()[0])))
    os.rename(f_province_zip.namelist()[0],f'province_ana{ana_date}.xls')

In [4]:
# unzip_file(ana_date)

## 清理当月数据

In [5]:
dbconn=cx_Oracle.connect(ana_db_user,ana_db_pass,ana_db)
dbcur=dbconn.cursor()
sql_stmt = 'DELETE FROM T_QSYWFZRB_RPT a WHERE a.report_mon = :ana_mon'
dbcur.execute(sql_stmt, [ana_mon])
dbcur.close()
dbconn.commit()
dbconn.close()

## 导入数据库表

In [6]:
province_kpi = pd.read_excel(f'全省业务发展日报({ana_date}).xls',header=None, sheet_name=None)
for sheet_num, sheet_key in enumerate(province_kpi.keys()):
    print(sheet_num, sheet_key)
    # 按Excel中的Sheet进行处理
    kpi_df = province_kpi[sheet_key]
    # 取分表的表头
    kpi_t_head_list = kpi_df[kpi_df[0]=='地市'].index
    # 取分表的表尾
    kpi_t_end_list = kpi_df[pd.eval("(kpi_df[0]=='全省') or (kpi_df[0]=='合计') or (kpi_df[0]=='江苏省')")].index
    print(kpi_t_head_list, kpi_t_end_list)
    for tab_num in range(len(kpi_t_head_list)):
        # 取内部明细表格
        tab = kpi_df[kpi_t_head_list[tab_num]:kpi_t_end_list[tab_num]+1].fillna(method='ffill').dropna(axis=1, how='all').fillna(method='ffill', axis=1)
        # 分解表格标题和表格数据内容
        tab_cap = tab[tab[0]=='地市']
        tab_content = tab[tab[0]!='地市']
        # 准备入库
        dbconn=cx_Oracle.connect(ana_db_user,ana_db_pass,ana_db)
        dbcur=dbconn.cursor()
        # 按列取值
        for tab_row in range(1, tab_cap.shape[1]):
            # 数据标题
            kpi_id = '{:02d}.{:02d}.{:02d}'.format(sheet_num,tab_num,tab_row)
            kpi_title = '|'.join([x.strip() for x in tab_cap[tab_row].drop_duplicates().to_list()])
            kpi_data = tab_content[tab_row].apply(
                lambda x:round(float(x.rstrip('%'))/100,4) if str(x)[-1]=='%' else (float('nan') if x == '-' else float(x)))
            # 处理NaN问题
            # print(kpi_data.to_list())
            sql_parm = [str(ana_date)[0:6], kpi_id, kpi_title]+[None if math.isnan(x) else x for x in kpi_data.to_list()]
            # print(len(kpi_data), sql_parm)
            if len(kpi_data) == 15:
                sql_stmt = '''INSERT INTO t_qsywfzrb_rpt(report_mon, kpi_id, kpi_name, c_01,c_02,c_03,c_04,c_05,c_06,c_07,c_08,c_09,c_10,c_11,c_12,c_13,c_15,c_99)
                VALUES(:report_mon, :kpi_id, :kpi_name, :c_01,:c_02,:c_03,:c_04,:c_05,:c_06,:c_07,:c_08,:c_09,:c_10,:c_11,:c_12,:c_13,:c_15,:c_99)'''
            elif len(kpi_data) == 14:
                sql_stmt = '''INSERT INTO t_qsywfzrb_rpt(report_mon, kpi_id, kpi_name, c_01,c_02,c_03,c_04,c_05,c_06,c_07,c_08,c_09,c_10,c_11,c_12,c_13,c_99)
                VALUES(:report_mon, :kpi_id, :kpi_name, :c_01,:c_02,:c_03,:c_04,:c_05,:c_06,:c_07,:c_08,:c_09,:c_10,:c_11,:c_12,:c_13,:c_99)'''
            else:
                print('Error, Please Check {}'.format(kpi_title))
                break
            # print(sql_parm)
            dbcur.execute(sql_stmt, sql_parm)
        dbcur.close()
        dbconn.commit()
        dbconn.close()

0 1、竞争态势
Int64Index([1, 20, 39, 58, 77, 96], dtype='int64') Int64Index([16, 35, 54, 73, 92, 111], dtype='int64')
1 2、用户发展
Int64Index([1, 20, 39], dtype='int64') Int64Index([16, 35, 55], dtype='int64')
2 3、流量经营
Int64Index([1, 20, 39, 59, 78, 98, 117], dtype='int64') Int64Index([16, 35, 55, 74, 94, 113, 131], dtype='int64')
3 4、终端发展
Int64Index([1, 20, 39, 58, 77, 96, 117, 136, 156, 175, 194, 215, 235, 254], dtype='int64') Int64Index([16, 35, 54, 73, 92, 113, 132, 152, 171, 190, 211, 231, 250, 269], dtype='int64')
4 5、家庭市场
Int64Index([1, 20, 39, 58], dtype='int64') Int64Index([16, 35, 54, 73], dtype='int64')
5 6、收入结构
Int64Index([1, 23, 43, 64, 85], dtype='int64') Int64Index([19, 39, 60, 81, 102], dtype='int64')
6 7、集团市场
Int64Index([  1,  22,  43,  63,  84, 103, 122, 141, 161, 179, 198, 216, 236,
            255, 273, 292, 311, 330, 350, 369, 388, 407, 426, 445, 464, 483,
            502, 520, 539, 558],
           dtype='int64') Int64Index([ 19,  40,  60,  81, 100, 119, 138, 158, 175, 195

## 按规则生成处理字段

In [7]:
# 读取当月字段
dbconn=cx_Oracle.connect(ana_db_user,ana_db_pass,ana_db)
dbcur=dbconn.cursor()
dbcur.execute('SELECT * FROM T_QSYWFZRB_RPT a WHERE a.REPORT_MON = {}'.format(str(ana_date)[:6]))
dbrlt=dbcur.fetchall()

df_ann_mon = pd.DataFrame(dbrlt)
# 读取原有配置规则
df_deal = pd.read_json('kpi_deal_cfg.json')
# 按规则计算，并入库
for i in df_deal.index:
    deal_rlt = eval(df_deal.loc[i,'kpi_deal_str'])
    deal_sql = """INSERT INTO t_qsywfzrb_rpt(report_mon, kpi_id, kpi_name, c_01,c_02,c_03,c_04,c_05,c_06,c_07,c_08,c_09,c_10,c_11,c_12,c_13,c_15,c_99)
                VALUES(:report_mon, :kpi_id, :kpi_name, :c_01,:c_02,:c_03,:c_04,:c_05,:c_06,:c_07,:c_08,:c_09,:c_10,:c_11,:c_12,:c_13,:c_15,:c_99)"""
    deal_sql_param = [ana_mon, df_deal.loc[i,'kpi_id'], df_deal.loc[i,'kpi_name']]+[None if math.isnan(x) else x for x in deal_rlt.to_list()]
    print(deal_sql_param)
    dbcur.execute(deal_sql, deal_sql_param)
dbcur.close()
dbconn.commit()
dbconn.close()

['202008', '99.01.01', '互联网电视销户|当月累计|当月离网率', 0.013462493737065095, 0.019252421289664645, 0.009054067475311013, 0.010011478528861424, 0.0067332806843619485, 0.007166282584014788, 0.009817251679406504, 0.0097383443745199, 0.009349681441482809, 0.010500136501774523, 0.007937477517833025, 0.011762180882970702, 0.00771484375, None, 0.010633350355828727]
['202008', '99.01.02', '当月累计(TB)|手机上网流量(TB)|其中5G网流量|DOU(G)', 3.8118274304418986, 3.7443710589362715, 3.877253046594982, 2.6538038291605304, 2.900850489662677, 2.2991533980582526, 2.925993220338983, 3.288691612903226, 3.5834117759693633, 2.7023989424983474, 3.7785500485908656, 2.6489288846520496, 3.399630571198713, None, 3.373736756023965]
['202008', '99.01.03', '总计|套餐内使用流量（TB）|DOU(G)', 4.695980066253115, 5.374075822410048, 4.919561375961625, 3.896970528899149, 5.707033313913458, 3.839724561650849, 3.9442989478099766, 4.83015776340989, 4.471317274510502, 4.5176951816936315, 4.209762390966259, 3.6941963472077832, 4.82776281144598, None, 4.7851

## 单独调整规则，并加入规则（不调整无需运行）

In [ ]:
# 测试处理字符串
deal_string = "df_ann_mon[df_ann_mon[2] == '其中：咪咕任我看|套餐内使用流量（TB）'].iloc[0,3:]/df_ann_mon[df_ann_mon[2] == '其中：咪咕任我看|当月到达'].iloc[0,3:]*1024"
eval(deal_string)

In [ ]:
# 处理字符串
print('目前配置项：\n', df_deal)

KPI_ID = input('KPI_ID:')
KPI_NAME = input('KPI_NAME:')
deal_string = input('deal_string:')

# 生成单独处理的规则
kpi_deal_rule = pd.DataFrame([[KPI_ID, 
                               KPI_NAME, 
                               deal_string,
                               np.nan]], 
                             columns = ["kpi_id", "kpi_name", "kpi_deal_str", "remark"])
# 合并规则
df_deal = df_deal.append(kpi_deal_rule, ignore_index=True,)
# 删除重复项
df_deal.drop_duplicates(inplace=True)

# 保存配置项
df_deal.to_json('kpi_deal_cfg.json', force_ascii=False)

## 生成最终结果Excel

In [8]:
# 读取当月字段
dbconn=cx_Oracle.connect(ana_db_user,ana_db_pass,ana_db)
dbcur=dbconn.cursor()
# 调用处理存储过程
dbcur.callproc('p_deal_qskb@nt_zhuangbin.ntdb3', [ana_mon])
# 获取全量数据
dbcur.execute('SELECT * FROM T_QSYWFZRB_RPT a WHERE a.REPORT_MON = {} order by a.kpi_id'.format(str(ana_date)[:6]))
dbrlt=dbcur.fetchall()
df_ana_mon = pd.DataFrame(dbrlt)
dbcur.close()
dbconn.commit()
dbconn.close()
df_ana_mon.to_excel(f'{ana_mon}月全量.xlsx', header=False, index=False)

<font color = 'red'>处理结束，下载结果数据并粘贴即可</font>

In [ ]:
dbcur.close()
dbconn.commit()
dbconn.close()

In [ ]:
str(ana_date)[0:6]

In [ ]:
kpi_df0 = province_kpi['7、集团市场']
kpi_df0

In [ ]:
#取分表的表头
kpi_t_head_list = kpi_df0[kpi_df0[0]=='地市'].index
#取分表的表尾
kpi_t_end_list = kpi_df0[pd.eval("(kpi_df0[0]=='全省') or (kpi_df0[0]=='合计')")].index

In [ ]:
kpi_t_head_list, kpi_t_end_list

In [ ]:
# 取内部明细表格
tab = kpi_df0[273:289+1].fillna(method='ffill').dropna(axis=1, how='all').fillna(method='ffill', axis=1)

In [ ]:
# 分解表格标题和表格数据内容
tab_cap = tab[tab[0]=='地市']
tab_content = tab[tab[0]!='地市']

In [ ]:
tab_cap

In [ ]:
# 数据标题
'|'.join(tab_cap[8].drop_duplicates().to_list())

In [ ]:
tab_cont_with_index = tab_content.set_index([0])

In [ ]:
kpi_data = tab_cont_with_index[8].apply(lambda x:round(float(x.rstrip('%'))/100,4) if str(x)[-1]=='%' else (None if x == '-' else x))

In [ ]:
kpi_data.to_list()

In [ ]:
[None if math.isnan(x) else x for x in a]

In [ ]:
b=float('nan')

In [ ]:
math.isnan(a[13])

In [ ]:
sql_parm = [202005,'1.2.3.4','Test']+kpi_data.to_list()

In [ ]:
dbconn=cx_Oracle.connect(ana_db_user,ana_db_pass,ana_db)

In [ ]:
dbcur=dbconn.cursor()

In [ ]:
#sql_stmt = '''INSERT INTO t_qsywfzrb_rpt(report_mon, kpi_id, kpi_name, c_01,c_02,c_03,c_04,c_05,c_06,c_07,c_08,c_09,c_10,c_11,c_12,c_13,c_15,c_99)
#VALUES(:report_mon, :kpi_id, :kpi_name, :c_01,:c_02,:c_03,:c_04,:c_05,:c_06,:c_07,:c_08,:c_09,:c_10,:c_11,:c_12,:c_13,:c_15,:c_99)'''

sql_stmt = '''INSERT INTO t_qsywfzrb_rpt(report_mon, kpi_id, kpi_name, c_01,c_02,c_03,c_04,c_05,c_06,c_07,c_08,c_09,c_10,c_11,c_12,c_13,c_99)
VALUES(:report_mon, :kpi_id, :kpi_name, :c_01,:c_02,:c_03,:c_04,:c_05,:c_06,:c_07,:c_08,:c_09,:c_10,:c_11,:c_12,:c_13,:c_99)'''

dbcur.execute(sql_stmt, sql_parm)
#dbrlt=dbcur.fetchall()
print(dbrlt)

In [ ]:
dbcur.close()
dbconn.commit()
dbconn.close()

In [ ]:
a = [' 123 ',' 456 ']

In [ ]:
[x.strip() for x in a]

In [ ]:
a[3].apply(lambda x:x if str(x)[-1]!='%' else float(x.rstrip('%'))/100)

In [ ]:
# 行数
print(kpi_df0.shape[0])
# 列数
print(kpi_df0.shape[1])